# LangChain: Evaluation

## Create our QandA application

In [6]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [7]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [8]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="all-minilm-l6-v2:latest")

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

d:\project\backend\codespaces-jupyter\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="phi3:3.8b", temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [10]:
data[2]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 2}, page_content=": 2\nname: Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece\ndescription: She'll love the bright colors, ruffles and exclusive whimsical prints of this toddler's two-piece swimsuit! Our four-way-stretch and chlorine-resistant fabric keeps its shape and resists snags. The UPF 50+ rated fabric provides the highest rated sun protection possible, blocking 98% of the sun's harmful rays. The crossover no-slip straps and fully lined bottom ensure a secure fit and maximum coverage. Machine wash and line dry for best results. Imported.")

## QAGenerateChain

In [11]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [12]:
from langchain.evaluation.qa import QAGenerateChain
from langchain_ollama import ChatOllama

example_gen_chain = QAGenerateChain.from_llm(ChatOllama(model="phi3:3.8b", temperature = 0.0))

In [15]:
len(data)

4

In [23]:
data[:5]

[Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries."),
 Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 1}, page_content=': 1\nname: Recycled Waterhog Dog Ma

In [16]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:2]]
)

C:\Users\jackc\AppData\Local\Temp\ipykernel_5968\1284517177.py:1: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  new_examples = example_gen_chain.apply_and_parse(


In [17]:
new_examples[0]

{'qa_pairs': {'query': "What is the weight of a pair of Women's Campside Oxfords?",
  'answer': "A pair of Women' end with an approximate weight of 1 lb. and 1 oz. per shoe."}}

In [18]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

## Combine examples

In [ ]:
examples += new_examples

In [21]:
qa.invoke(examples[0]['query'])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': "I'm sorry, but I don't have that information based on the context provided. If you can provide more details about this product or check its specifications directly from a reliable source like an official website or catalogue, it should be possible to find out if there are side pockets included in the Cozy Comfort Pullover Set."}

In [19]:
new_examples[1]

{'qa_pairs': {'query': 'What are the dimensions of the Recycled Waterhog Dog Mat in its medium size?',
  'answer': 'The Medium-sized Recycled Waterhog Dog Mat has dimensions of 22.5 inches by 34.5 inches.'}}

In [20]:
data[1]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 1}, page_content=': 1\nname: Recycled Waterhog Dog Mat, Chevron Weave\ndescription: Protect your floors from spills and splashing with our ultradurable recycled Waterhog dog mat made right here in the USA. \r\n\r\nSpecs\r\nSmall - Dimensions: 18" x 28". \r\nMedium - Dimensions: 22.5" x 34.5".\r\n\r\nWhy We Love It\r\nMother nature, wet shoes and muddy paws have met their match with our Recycled Waterhog mats. Ruggedly constructed from recycled plastic materials, these ultratough mats help keep dirt and water off your floors and plastic out of landfills, trails and oceans. Now, that\'s a win-win for everyone.\r\n\r\nFabric & Care\r\nVacuum or hose clean.\r\n\r\nConstruction\r\n24 oz. polyester fabric made from 94% recycled materials.\r\nRubber backing.\r\n\r\nAdditional Features\r\nFeatures an -exclusive design.\r\nFeatures thick and thin fibers for scraping dirt and absorbing water.\r\nDries quickly and resists fadi

## Manual Evaluation

In [24]:
import langchain
langchain.debug = True

In [25]:
qa.run(examples[0]["query"])

C:\Users\jackc\AppData\Local\Temp\ipykernel_5968\1223946598.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])


[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 2\nname: Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece\ndescription: She'll love the bright colors, ruffles and exclusive whimsical prints of this toddler's two-piece swimsuit! Our four-way-stretch and chlorine-resistant fabric keeps its shape and resists snags. The UPF 50+ rated fabric provides the highest rated sun protection possible, blocking 98% of the sun's harmful rays. The crossover no-slip straps and fully lined bottom ensure a secure fit and maximum coverage. Machine wash and line dry for best results. Imported.<<<<>>>>>: 3

"I'm sorry, but I don't have that information based on the context provided. If you can provide more details about this product or check its specifications directly from a reliable source like an official website or catalogue, it should be possible to find out if there are side pockets included in the Cozy Comfort Pullover Set."

In [26]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [43]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.


In [44]:
from langchain.evaluation.qa import QAEvalChain

In [45]:
llm = ChatOllama(model="phi3:3.8b", temperature = 0.0)
eval_chain = QAEvalChain.from_llm(llm)

In [46]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [48]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: I'm sorry, but I don't have that information based on the context provided. If you can provide more details about this product or check its specifications directly from a reliable source like an official website or catalogue, it should be possible to find out if there are side pockets included in the Cozy Comfort Pullover Set.

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The information provided does not specify a particular collection for the "Ultra-Lofty 850 Stretch Down Hooded Jacket." To find out which collection it belongs to, you may need to refer directly to the product's packaging or contact customer service of the retailer from where you purchased or intend to purchase this item.

